# Création d'un scatter plot (plus) interactif

Nous allons maitenant explorer la librarie [ipywidget](https://ipywidgets.readthedocs.io/en/latest/index.html).
Elle permet la construction d'un interface utilisateur dans les cellules du notebook. Elle peut prendre en charge:
* [La création d'éléments](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)(les *widgets*) comme un bouton ou un slider
* [L'organisation des éléments](https://ipywidgets.readthedocs.io/en/latest/examples/Layout%20Templates.html) dans la cellule d'affichage de Jupyter
* L'expression de lien entres éléments et figure matplotlib pour permettre d'interagire avec ces images.

L'objectif est de fournir ici une introduction à ce module. Nous nous limiterons donc à l'utilisation de la fonction [interact](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#) pour interagir avec le scatter plot précedent.


In [2]:
import sys

rootDir = "//ifb/data/mydatalocal"
projectDir = rootDir + "/VDBscratch"
libDir = projectDir + "/lib"
dataDir= "/ifb/data/public/teachdata/m2binf_ucbl/data" 

sys.path.append(libDir)

In [3]:
import pandas, uniprot, go
import numpy as np
from stat_utils import computeORA_BKG
%matplotlib widget

import matplotlib.pyplot as plt

#### Visualisation des abondances et Pathway pour chaque protéine
Nous allons représenter la relation entre l'abondance des protéines (**Log2 Corrected Abundance Ratio**)  et la précision de la mesure (**LOG10 Adj.P-val**). 
###### Creation du scatter plot
[matplotlib.axes.Axes.scatter()](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.scatter.html) permet d'intialiser le graphique.
###### Modification continue du scatter plot
Mais cette fonction retourne aussi un objet [PathCollection](https://matplotlib.org/3.1.1/api/collections_api.html) qui permet de modifier à posteriori le rendu des points du scatter plot.
###### Interactivité
Elle sera ici prise en charge par la fonction interact dont le fonctionnement est illustré ci-dessous.



### Exemple de widgets

#### widget simple

In [4]:
import ipywidgets as widgets
from IPython.display import display
w = widgets.IntSlider()
display(w)

IntSlider(value=0)

#### widgets combinés

In [5]:
a = widgets.FloatText()
b = widgets.FloatSlider()
display(a,b)

mylink = widgets.jslink((a, 'value'), (b, 'value'))

FloatText(value=0.0)

FloatSlider(value=0.0)

#### Inférence du widget avec interact
Jupyter construit le widget adapté au paramètre de la fonction interact. Ici un nombre *w* flottant. On va spécifier la modification à apporter à la figure dans le corps de la fonction *update*.

In [6]:
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 2 * np.pi)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(x, np.sin(x))

def update(w = 1.0):
    line.set_ydata(np.sin(w * x))
    fig.canvas.draw_idle()

interact(update);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='w', max=3.0, min=-1.0), Output()), _dom_classes=('wi…

Jupyter construit le widget adapté au paramètre de la fonction interact. **Ici une liste**.

In [7]:
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt

#%matplotlib notebook

x = np.linspace(0, 2 * np.pi)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(x, np.sin(x))

def update(w):
    line.set_ydata(np.sin(w * x))
    fig.canvas.draw_idle()

#interact(update);
interact(update, w=[1,3,6]);


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='w', options=(1, 3, 6), value=1), Output()), _dom_classes=('widget-…

Pour plus de contrôle, un widget peut être créé explicitement et passé à la fonction interact, **ici un Dropdown**

In [8]:
listDropW = widgets.Dropdown(
    options=[1, 2, 3],
    value=2,
    description='Number:',
    disabled=False,
)

x = np.linspace(0, 2 * np.pi)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(x, np.sin(x))

def update(w):
    line.set_ydata(np.sin(w * x))
    fig.canvas.draw_idle()

interact(update, w=listDropW);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='Number:', index=1, options=(1, 2, 3), value=2), Output()), _dom_cl…

## Volcano plot interactif

Nous pouvons desormais construire un volcano plot interactif dans lequel:
* un widget contiendra la liste des pathway
* une interaction avec la liste mettra en évidence les protéines de ce pathway

#### Préparation des données

Réutiliser le travail du **TP3**.

* Lecture des données pandas.
    1. Filtrage des protéines quantifiées
    2. Assignation à chaque protéine des 'Log2 Corrected Abundance Ratio' et 'LOG10 Adj.P-val'
* Lecture de l'ontologie GO.
    1. Lecture des données protéome entier.
    2. Analyse ORA sur l'arbre de termes de votre choix 
    3. Classement des termes GO par pValue croissante
    4. Pour chacun des n termes GO ainsi classé, extraction des protéines de son sous-arbre.
   
   ###### OU
    Deserialisation des données du TP2 `TP_ORA.json`

##### Pour chaque proteine on obiendra donc
* *Log2 Corrected Abundance Ratio*
* *LOG10 Adj.P-val*
* [Pathway GO]

In [13]:
import pandas
df=pandas.read_csv(projectDir + "/TCL_wt1.tsv", delimiter ="\t")

dfClean = df[ (df['Log2 Corrected Abundance Ratio'] != '#VALEUR!') & (df['LOG10 Adj.P-val'] != '#VALEUR!')]
dfClean = dfClean.copy()
dfClean.loc[:,'Log2 Corrected Abundance Ratio'] = dfClean.loc[:,'Log2 Corrected Abundance Ratio'].astype(float)
dfClean.loc[:,'LOG10 Adj.P-val'] = dfClean.loc[:,'LOG10 Adj.P-val'].astype(float)


saList = dfClean[ dfClean['Log2 Corrected Abundance Ratio'] > 0.0 ]['Accession'].tolist()
allList = dfClean[ : ]['Accession'].tolist()
pvalue = dfClean[ : ]['LOG10 Adj.P-val'].tolist()
abundance = dfClean[ : ]['Log2 Corrected Abundance Ratio'].tolist()

In [14]:
import json
pathData = {}
with open('/ifb/data/mydatalocal/VDB_jupyter/solutions/TP_ORA.json', 'r') as fp:
    pathData = json.load(fp)

x, y, label = (abundance, pvalue, allList)

In [19]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
col = ax1.scatter(x, y)
pathwayList = list(pathData.keys())

def getColor(name, pathSel):
    if name in pathData[pathSel]:
        return 'tab:red'
    return 'tab:gray'

def update(w):
   # Get selected pathway
    pathSel = w
   # Generate proper point color vector 
    newColors = [ getColor(name, pathSel) for name in label ]
    col.set_color( newColors )

   # Generate proper point sizes vector
   #  col.set_sizes( )
    pass

interact(update, w=pathwayList)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='w', options=('ion transmembrane transport', 'L-glutamate transmemb…

<function __main__.update(w)>

RuntimeError: libpng signaled error

In [16]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
fig = plt.figure()
ax1 = fig.add_subplot(111)
col = ax1.scatter(x, y)

display(button)
def on_button_clicked(b):
    col.set_color('tab:gray')

button.on_click(on_button_clicked)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Click Me!', style=ButtonStyle())

In [17]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
col = ax1.scatter(x, y)
pathwayList = list(pathData.keys())

wSel = widgets.SelectMultiple(
    options=pathwayList,
    description='GO pathways',
    disabled=False
)

def getColorMult(name, pathSelList):
    for pathSel in pathSelList:
        if name in pathData[pathSel]:
            return 'tab:red'
    return 'tab:gray'

def update(w):
    pathSelList = w
    # Get selected pathway
   # Generate proper point color vector 
    newColors = [ getColorMult(name, pathSelList) for name in label ]
    col.set_color( newColors )
   # Generate proper point sizes vector
   #  col.set_sizes( )
    
interact(update, w=wSel)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(SelectMultiple(description='GO pathways', options=('ion transmembrane transport', 'L-glu…

<function __main__.update(w)>

In [18]:
x,y, labels = (abundance, pvalue, allList)
fig = plt.figure()
ax1 = fig.add_subplot(111)
col = ax1.scatter(x, y)


dType = None


pathways = list(pathData.keys())

def getColor(name, pathway_label):
    #print(name)
    #print(pathway_label)
    if name in pathData[pathway_label]:
        return 'tab:red'
    else:
        return 'tab:grey'

def getSize(name, pathway_label, min_v = 10, max_v = 100):
    if name in pathData[pathway_label]:
        return max_v
    else:
        return min_v

wid_button = widgets.ToggleButtons(
    options=['Color', 'Size', 'Both'],
    description='Representation',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

wid_dropdown = widgets.Dropdown(
    options=pathways,
    description='Pathway:',
    disabled=False,
)

def updateDropdown(w):
    pathway_label =  w
    newcolors = [ getColor(name, pathway_label) for name in labels ]
    newsize = [ getSize(name, pathway_label) for name in labels ]
    col.set_color(newcolors)
    col.set_sizes(newsize)

def updateToggle(w):
    dType = w
    print(dType)
    
interact(updateDropdown,w=wid_dropdown)
interact(updateToggle, w=wid_button)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='Pathway:', options=('ion transmembrane transport', 'L-glutamate tr…

interactive(children=(ToggleButtons(description='Representation', options=('Color', 'Size', 'Both'), tooltips=…

<function __main__.updateToggle(w)>

In [21]:
dd

NameError: name 'create_expanded_button' is not defined